## Training the classifier

Use this notebook to train the Image3C classifier using the clusters assigned previously
as labels for classes.

Before running this notebook, the [Image preprocessing](image_preprocessing.ipynb) notebook needs to be run
to save the images for training into the right format and to create a file with
the cluster IDs as labels from the clustering results.

To perform the training we only need to import image3c.network_training.

This notebook creates a dictionary of parameters to tell the trainer where to
find images and labels, and where to save the trained network, and other network
parameters.



---

The source Python code for the Image3C classifier can be found on Github: [Image3C source](https://github.com/stowersinstitute/LIBPB-1390-Image3C/tree/master/Code/Classifier).

In [ ]:
import os

from image3c import network_training

In [ ]:
# get the operating system separator to use in paths to avoid file errors

slash = os.path.sep

### Create the parameter dictionary

- description :   Description of the experiment
- num_classes :   Number of clusters found with Vortex clustering
- datafile :      Location of the datafile in numpy format (from [imageprep.ipynb](imageprep.ipynb))
- labelsfile :    Location of the file with class labels (from [imageprep.ipynb](imageprep.ipynb))
- checkpointDir : Location to save intermediate and final models
- num_channels :  Number of channels in the dataset.
- channels :      List of channels to be used for training, i.e., [0, 1, 2] or [1,2,4], if only a subset of channels is going to be used
- iterations :   Number of iterations to perform (around 25,000 worked well)
- batchsize  :   Number of images in an iteration. This depends on the memory of the system or GPU. Start with 256 and decrease it if memory errors are experienced.
- combine   :   Cluster pairs that need to be combined. Use and empty list if not combining. 

#### Combining clusters
The `combine` parameter is used when two clusters with similar morphology have significant overlap in the Force Directed Layout (FDL) graph. The two clusters can be combined into one cluster with the final cluster ID of the lowest, e.g., if clusters 2 and 7 are combined, the resulting cluster will have 2 as cluster ID.


In [ ]:
params = dict()
params['description'] = "Training description" ## description for the training

## number of classes, typically the same as the number of clusters found previously in Vortex
num_classes = 7  

# path to the numpy file with images
params['datafile'] = 'Data/snail_images.npy'

# path to numpy file with cluster labels
params['labelsfile'] = 'Data/snail_labels.npy'

# path to checkpoints
params['CheckpointDir'] = "Checkpoints/test"

# the number of channels in the images
params['num_channels'] = 5

# channels to use. Start at zero!
params['channels'] = [0,2,4]

# channels to combine
params['combine'] = [[0,8], [4,7]] 

params['clusterlist'] = list(range(num_classes))


params['iterations'] = 25000 #25000
params['learning_rate'] = 0.001 #0.0006
params['droprate'] = 0.0
params['l2f'] = 0.006 #0.004
params['batchsize'] = 256
params['tensorboard_log_dir'] = 'logs'
### name of output checkpoint directory

if not params['CheckpointDir'].endswith(slash):
    params['CheckpointDir'] += slash



### Run the training

Now the network can be trained with `network_training.run()`.

#### Testing and validation
Ten percent of the images are held out for testing during the training and another ten percent are held out for validation after the training. The images and labels for validation are saved as a numpy files in the `CheckpointDir` folder defined above.

#### Normalization
The images for training are normalized to have zero mean and a standard deviation of one. It is important to perform this normalization on images to be predicted after training.

In [ ]:
network_training.run(params)

print("Done")

### Post-training

After the training is complete, model files called checkpoints are saved in the `CheckpointDir` defined in the parameters. A checkpoint is created for the training iteration with the best accuracy, and a checkpoint for the final training iteration. An example of using a checkpoint can be found in [Performance testing](performance_testing.ipynb).